# Init

In [1]:
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
from sympy import Matrix, Line, Ellipse, Point2D, Point3D
from matplotlib.animation import FuncAnimation, PillowWriter
from matplotlib.patches import Polygon
from mpl_toolkits.mplot3d import Axes3D

# Task 1

In [ ]:
def cl_intersection(O, r, A, B, C):
    C1 = C + A * O[0] + B * O[1]
    
    norm = (A**2 + B**2)
    d = r**2 - C1**2 / norm
    mult = sp.sqrt(d / norm)
    
    x0 = - A * C1 / norm
    y0 = - B * C1 / norm
    
    P1 = Point2D([x0 + B * mult + O[0], y0 - A * mult + O[1]])
    P2 = Point2D([x0 - B * mult + O[0], y0 + A * mult + O[1]])
    
    return (P1, P2)

In [ ]:
def c_intersection(O1, r1, O2, r2):
    
    A = -2 * (O2[0] - O1[0])
    B = -2 * (O2[1] - O1[1])
    C = (O2[0] - O1[0])**2 + (O2[1] - O1[1])**2 + r1**2 - r2**2
    (P1, P2) = cl_intersection(Point2D(0, 0), r1, A, B, C)
    (P1, P2) = (Point2D([P1[0] + O1[0], P1[1] + O1[1]]), Point2D([P2[0] + O1[0], P2[1] + O1[1]]))
    return (P1, P2)

In [ ]:
phi0 = np.pi / 3
w_O1A = 2
a = 56
b = 10
c = 26
d = 16
e = 25
O1A = 21
O2B = 25
O3F = 20
AB = 54
AC = 23
BC = 52
CD = 69
CE = 35
EF = 32

In [ ]:
t = sp.Symbol('t')

O1 = sp.Point2D([0, 0])
O2 = sp.Point2D([a, -c])
O3 = sp.Point2D([a + b, d + e])

In [ ]:
A = np.asarray(O1A * sp.Point2D([sp.cos(w_O1A * t + phi0), sp.sin(w_O1A * t + phi0)]))
B = np.asarray(c_intersection(A, AB, O2, O2B)[0])
C = np.asarray(c_intersection(A, AC, B, BC)[0])
D = np.asarray(cl_intersection(C, CD, 0, 1, -d)[0])
E = CE / CD * (D - C) + C
F = np.asarray(c_intersection(O3, O3F, E, EF)[0])

V = [[sp.diff(A[0], t), sp.diff(A[1], t)], 
     [sp.diff(B[0], t), sp.diff(B[1], t)],
     [sp.diff(C[0], t), sp.diff(C[1], t)],
     [sp.diff(D[0], t), sp.diff(D[1], t)],
     [sp.diff(E[0], t), sp.diff(E[1], t)],
     [sp.diff(F[0], t), sp.diff(F[1], t)],]

In [ ]:
fig, ax = plt.subplots()
ax.set_aspect("equal")
xdata, ydata = [], []
ln, = plt.plot([], [], 'b-')

titles = []
points = []
lines = []
vecs = []

# animation initialisation
def init_task1():
    ax.set_xlim(-30, 120)
    ax.set_ylim(-60, 90)
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    plt.plot([-30, 120], [d, d], linestyle='--')
    return ln,

# animation update on every frame
def update_task1(frame):
    # substitute frame for time variable
    # in sympy objects
    O1_frame = O1
    O2_frame = O2
    O3_frame = O3

    try:
        A_frame = np.asarray([float(i.subs(t, frame)) for i in A])
        B_frame = np.asarray([float(i.subs(t, frame)) for i in B])
        C_frame = np.asarray([float(i.subs(t, frame)) for i in C])
        D_frame = np.asarray([float(i.subs(t, frame)) for i in D])
        E_frame = np.asarray([float(i.subs(t, frame)) for i in E])
        F_frame = np.asarray([float(i.subs(t, frame)) for i in F])
    except:
        ln.set_data(xdata, ydata)
        return ln,
    
    V_frame = np.asarray([
        [float(i[0].subs(t, frame)), float(i[1].subs(t, frame))]
        for i in V
    ])

    a_frame = np.asarray([
        [float(sp.diff(i[0], t).subs(t, frame)), float(i[1].subs(t, frame))]
        for i in V[0:2]
    ])

    a_t_frame = np.asarray([
        np.dot(a, v) / np.linalg.norm(v) ** 2 * v for a, v in zip(a_frame, V_frame[0:2])
    ])

    a_n_frame = np.asarray([
        a - at for a, at in zip(a_frame, a_t_frame)
    ])
    
    P = [O1_frame, 
         O2_frame, 
         O3_frame, 
         A_frame, 
         B_frame, 
         C_frame, 
         D_frame, 
         E_frame, 
         F_frame]
    #B_test = circle_intersection(A_frame, AB, O2_frame, O2B)[0]
    #B_test = np.asarray([float(B_test[0]), float(B_test[1])])
    #print(np.linalg.norm(A_frame - C_frame) - AC, np.linalg.norm(B_test - C_frame) - BC)

    while len(points):
        points[-1][0].remove()
        points.pop()
        
    while len(lines):
        lines[-1][0].remove()
        lines.pop()
        
    while len(titles):
        titles[-1].remove()
        titles.pop()
        
    while len(ax.patches) > 1:
        ax.patches[-1].remove()
        ax.patches.pop()
        
    while len(vecs):
        vecs[-1].remove()
        vecs.pop()
    
    p1, p2 = Ellipse(O1, O1A, O1A).intersection(Ellipse(O2, AB + O2B, AB + O2B))
    ax.add_patch(plt.Circle(O1, O1A, fill=False, linestyle='--'))
    ax.add_patch(plt.Circle(O2, O2B, fill=False, linestyle='--'))
    ax.add_patch(plt.Circle(O3, O3F, fill=False, linestyle='--'))
    ax.add_patch(plt.Circle(O2, 79, fill=False, linestyle='--'))
    ax.add_patch(Polygon([A_frame, B_frame, C_frame], facecolor='red', fill=False, hatch='//'))
    ax.add_patch(plt.Circle(O1, 0, fill=False, linestyle='--'))
    ax.add_patch(plt.Circle(O1, 0, fill=False, linestyle='--'))
    
    lines.extend([
        plt.plot([A_frame[0], O1_frame[0]], [A_frame[1], O1_frame[1]], color='green'),
        plt.plot([O2_frame[0], B_frame[0]], [O2_frame[1], B_frame[1]], color='green'),
        plt.plot([C_frame[0], D_frame[0]], [C_frame[1], D_frame[1]], color='black'),
        plt.plot([F_frame[0], E_frame[0]], [F_frame[1], E_frame[1]], color='black'),
        plt.plot([F_frame[0], O3_frame[0]], [F_frame[1], O3_frame[1]], color='green'),
        
    ])
    
    points.extend([
        plt.plot(i[0], i[1], marker="o", color='red', markersize=3)
        for i in list(P) + [p1, p2]
    ])
    
    titles.extend([
        ax.text(x, y + 3, title)
        for [x, y], title in zip(P, [r'$O_1$', r'$O_2$', r'$O_3$', 
                                'A', 'B', 'C', 'D', 'E', 'F'])
    ])

    vecs.extend([
        plt.quiver(x, y, u, v, color="blue", units='xy', scale = 2, scale_units='xy', angles='xy')
        for [x, y], [u, v] in zip(P[3:], V_frame)
    ])

    vecs.extend([
        plt.quiver(x, y, u, v, color="green", units='xy', scale = 2, scale_units='xy', angles='xy')
        for [x, y], [u, v] in zip(P[3:5], a_frame[0:2])
    ])

    vecs.extend([
        plt.quiver(x, y, u, v, color="red", units='xy', scale = 2, scale_units='xy', angles='xy')
        for [x, y], [u, v] in zip(P[3:5], a_t_frame[0:2])
    ])

    vecs.extend([
        plt.quiver(x, y, u, v, color="yellow", units='xy', scale = 2, scale_units='xy', angles='xy')
        for [x, y], [u, v] in zip(P[3:5], a_n_frame[0:2])
    ])

    titles.extend(
        ax.text(x + u / 2, y + v / 2, title, size=8)
        for [x, y], [u, v], title in zip(P[3:5], a_frame[0:2], [r'$\vec{a}_A$', r'$\vec{a}_B$'])
    )

    titles.extend(
        ax.text(x + u / 2, y + v / 2, title, size=8)
        for [x, y], [u, v], title in zip(P[3:5], a_t_frame[0:2], [r'$\vec{a}^{\tau}_A$', r'$\vec{a}^{\tau}_B$'])
    )

    titles.extend(
        ax.text(x + u / 2, y + v / 2, title, size=8)
        for [x, y], [u, v], title in zip(P[3:5], a_n_frame[0:2], [r'$\vec{a}^n_A$', r'$\vec{a}^n_B$'])
    )
    
    titles.extend([
        ax.text(x + u / 2, y + v / 2, title, size=8)
        for [x, y], [u, v], title in zip(P[3:], V_frame, 
                                         [r'$\vec{V_A}$', r'$\vec{V_B}$', 
                                          r'$\vec{V_C}$', r'$\vec{V_D}$', 
                                          r'$\vec{V_E}$', r'$\vec{V_F}$'])
    ])
    
    ln.set_data(xdata, ydata)
    return ln,

p1, p2 = Ellipse(O1, O1A, O1A).intersection(Ellipse(O2, AB + O2B, AB + O2B))
t_start, t_end = float(sp.solve(A - p1, t)[0][0].evalf()), float(sp.solve(A - p2, t)[0][0].evalf())
t_start = t_start - 2 * np.pi / w_O1A
eps = 0.1
frames = np.linspace(t_start + eps, t_end - eps, 120)

# create animation
anim = FuncAnimation(fig, update_task1, frames=frames, init_func=init_task1, blit=True)

# save animation to the file
anim.save('task1.gif', dpi=100, writer=PillowWriter(fps=60))
# prevent unclosed plots
plt.close('all')

In [ ]:
p1, p2 = Ellipse(O1, O1A, O1A).intersection(Ellipse(O2, AB + O2B, AB + O2B))


In [ ]:
t1 = sp.solve(A - p1, t)
t1[0][0].evalf()

In [ ]:
t2 = sp.solve(A - p2, t)
t2[0][0].evalf()

In [ ]:
np.sqrt(56**2 + 26**2)

In [ ]:
V_frame = [[0, 0], [0, 0], [0, 0]] + [
        [float(i[0].subs(t, 0.0001)), float(i[1].subs(t, 0.0001))]
        for i in V
    ]

In [ ]:
[O1_V, O2_V, O3_V, A_V, B_V, C_V, D_V, E_V, F_V] = V_frame

In [ ]:
def eps_AB(frame):
    A_V = np.asarray(V[0])
    B_V = np.asarray(V[1])
    w = sp.sqrt(np.dot(A_V - B_V, A_V - B_V)) / AB
    e = sp.diff(w)
    e = e.subs(t, frame).evalf()
    return e

In [ ]:
eps_AB(0.001)

In [ ]:
for [P1_V, P2_V], [r, title] in zip([[O1_V, A_V], [O2_V, B_V], [O3_V, F_V],
                    [A_V, B_V], [A_V, C_V], [B_V, C_V], 
                    [C_V, E_V], [C_V, D_V], [E_V, D_V], 
                    [E_V, F_V]], 
                   [[O1A, 'O1A'], [O2B, 'O2B'], [O3F, 'O3F'],
                    [AB, 'AB'], [AC, 'AC'], [BC, 'BC'], 
                    [CE, 'CE'], [CD, 'CD'], [CD - CE, 'ED'], 
                    [EF, 'EF']]):
    print(title + " =", np.linalg.norm([P1_V[0] - P2_V[0], P1_V[1] - P2_V[1]]) / r)


# Task 2

In [2]:
"""
A function to plot a truncated cone
with a given axis and a radii.
Returns plotted object.
"""
def truncated_cone(p0, p1, R0, R1, ax):
    v = p1 - p0                           # Axis vector
    mag = np.linalg.norm(v)               # Norm of axis vector
    v = v / mag                           # Unit axis vector
    not_v = np.array([1, 1, 0])           # Noncollinear vector
    if (v == not_v).all():
        not_v = np.array([0, 1, 0])
    n1 = np.cross(v, not_v)               # Second axis
    n1 /= np.linalg.norm(n1)              # Second unit axis
    n2 = np.cross(v, n1)                  # Third unit axis 
    t = np.linspace(0, mag, 80)           # Magnitude split
    theta = np.linspace(0, 2 * np.pi, 80) # Angle split
    t, theta = np.meshgrid(t, theta)      # Magnitute and radius mesh
    R = np.linspace(R0, R1, 80)           # Radius split
    X, Y, Z = [p0[i] + v[i] * t 
                + R * np.sin(theta) * n1[i] 
                + R * np.cos(theta) * n2[i] for i in [0, 1, 2]]

    return ax.plot_wireframe(X, Y, Z, rstride=7, cstride=7, alpha=0.5)

In [3]:
# TASK PREDEFINED VALUES
OA = OM0 = 40
M0M = 5
w1 = 2
e1 = 3.7
ro1 = OA / 2 * np.sin(np.pi / 6) / np.sin(75 / 180 * np.pi)

OO1 = np.sqrt(40**2 - ro1**2)
OO2 = OO1 / np.sqrt(2)
r = 10
R = OO1 / np.sqrt(2) 

In [4]:
# FUNCTIONS DEFINITIONS
t = sp.Symbol('t')
e1_t = 3.7
w1_t = 3.7 * t + 2
phi1_t = 1.85 * t**2 + 2 * t
phi2_t = phi1_t * OO1/np.sqrt(2) / 10

In [5]:
print(OO1, OO2)

38.63703305156273 27.32050807568877


In [6]:
# Find the function of the second cone axis
O2 = np.asarray(Point3D([OO1 * sp.sin(phi1_t), OO1 * sp.cos(phi1_t), OO2]))

In [7]:
# Find axes of the cone
temp_O2 = np.dot(O2, sp.rot_axis3(np.pi / 6))
n1 = np.asarray(O2)               # First axis
n2 = np.cross(O2, temp_O2)        # Second axis
n2 = n2 / sp.sqrt(np.dot(n2, n2)) # Second unit axis
n3 = np.cross(n1, n2)             # Third axis
n3 = n3 / sp.sqrt(np.dot(n3, n3)) # Third unit axis

In [15]:
# Express M and M0 in a stationary coordinate system
M0 = np.asarray([O2[i] + ro1 * sp.cos(phi2_t) * n2[i] + ro1 * sp.sin(phi2_t) * n3[i] for i in [0, 1, 2]])
M = M0M / r * (O2 - M0) + M0

In [9]:
# Velocity of M
V = np.array([sp.diff(i, t) for i in M])
A_M = np.array([sp.diff(i, t) for i in V])

In [16]:
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1, projection='3d')

# Model objects
lines = []
points = []
vecs = []
cones = []
titles = []

# Stationary cone Z axis
A0 = np.array([0, 0, 0])
A1 = np.array([0, 0, OA / 2])



# Define the dimensions and plot the cone
def init_cmodel():
    ax.set_xlim(-25, 25)
    ax.set_ylim(-25, 25)
    ax.set_zlim(0, 50)
    ax.axis('off')
    ax.grid(False)
    truncated_cone(A0, A1, 0, np.sqrt(OA**2 - 20**2), ax)
    return []

# Clear model objects
def clear_model():
    while len(lines):
        lines[-1][0].remove()
        lines.pop()

    while len(points):
        points[-1].remove()
        points.pop()

    while len(cones):
        cones[-1].remove()
        cones.pop()
        
    while len(vecs):
        vecs[-1].remove()
        vecs.pop()
        
    while len(titles):
        titles[-1].remove()
        titles.pop()

# Frame by frame update of the model
def update_cmodel(frame):
    # Substitute frame for t in functions
    O2_frame = np.asarray([float(i.subs(t, frame)) for i in O2])
    #n1_frame = np.asarray([float(i.subs(t, frame)) for i in n1])
    #n2_frame = np.asarray([float(i.subs(t, frame)) for i in n2])
    #n3_frame = np.asarray([float(i.subs(t, frame)) for i in n3])
    M0_frame = np.asarray([float(i.subs(t, frame)) for i in M0])
    M_frame = np.asarray([float(i.subs(t, frame)) for i in M])
    V_frame = np.asarray([float(i.subs(t, frame)) for i in V])
    A_M_frame = np.asarray([float(i.subs(t, frame)) for i in A_M])

    A_t = np.dot(A_M_frame, V_frame) / np.linalg.norm(V_frame) * V_frame / np.linalg.norm(V_frame)
    A_n = A_M_frame - A_t

    a_scale = 0.003
    v_scale = 0.03

    clear_model()

    # Plot the second cone
    cones.append(truncated_cone(A0, O2_frame, 0, ro1, ax))
    
    # Plot the M point
    points.extend([
        ax.scatter(M_frame[0], M_frame[1], M_frame[2], color='red', linewidth=1),
        ax.scatter(M0_frame[0], M0_frame[1], M0_frame[2], color='red', linewidth=1),
        ax.scatter(O2_frame[0], O2_frame[1], O2_frame[2], color='red', linewidth=1),
    ])
    
    # Plot the lines for clearer picture
    lines.extend([ax.plot([M0_frame[0], O2_frame[0]], [M0_frame[1], O2_frame[1]], [M0_frame[2], O2_frame[2]], color='blue', alpha=0.5),
                 #ax.plot([M0_frame[0], A0[0]], [M0_frame[1], A0[1]], [M0_frame[2], A0[2]], color='green'),
                 ax.plot([A0[0], O2_frame[0]], [A0[1], O2_frame[1]], [A0[2], O2_frame[2]], color='blue', alpha=0.5),
    ])
    
    # Plot vectors
    vecs.extend([
        ax.quiver(M_frame[0], M_frame[1], M_frame[2], V_frame[0], V_frame[1], V_frame[2], color='black', length=v_scale),
        ax.quiver(M_frame[0], M_frame[1], M_frame[2], A_M_frame[0], A_M_frame[1], A_M_frame[2], color='green', length=a_scale),
        ax.quiver(M_frame[0], M_frame[1], M_frame[2], A_t[0], A_t[1], A_t[2], color='red', length=a_scale),
        ax.quiver(M_frame[0], M_frame[1], M_frame[2], A_n[0], A_n[1], A_n[2], color='blue', length=a_scale),
    ])
    
    # Plot titles
    titles.extend([
        ax.text(M_frame[0] + V_frame[0] * v_scale, M_frame[1] + V_frame[1] * v_scale, M_frame[2] + V_frame[2] * v_scale, r'$\vec{V_M}$', fontsize=12),
        ax.text(M_frame[0] + A_M_frame[0] * a_scale, M_frame[1] + A_M_frame[1] * a_scale, M_frame[2] + A_M_frame[2] * a_scale, r'$\vec{a}$', fontsize=12),
        ax.text(M_frame[0] + A_t[0] * a_scale, M_frame[1] + A_t[1] * a_scale, M_frame[2] + A_t[2] * a_scale, r'$\vec{a_t}$', fontsize=12),
        ax.text(M_frame[0] + A_n[0] * a_scale, M_frame[1] + A_n[1] * a_scale, M_frame[2] + A_n[2] * a_scale, r'$\vec{a_n}$', fontsize=12)
    ])
    
    return []


# Generate animation
anim = FuncAnimation(fig, update_cmodel, frames=np.linspace(0, 4, 500),
                        init_func=init_cmodel, blit=True)

anim.save('task2.gif', dpi=300, writer=PillowWriter(fps=60))
plt.close('all')

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1, projection='3d')

# Model objects
lines = []
points = []
vecs = []
cones = []
titles = []

# Stationary cone Z axis
A0 = np.array([0, 0, 0])
A1 = np.array([0, 0, OA / 2])



# Define the dimensions and plot the cone
def init_cmodel():
    ax.set_xlim(-10, 10)
    ax.set_ylim(-10, 10)
    ax.set_zlim(-10, 10)
    ax.axis('off')
    #ax.grid(False)
    #truncated_cone(A0, A1, 0, R, ax)
    return []

# Clear model objects
def clear_model():
    while len(lines):
        lines[-1][0].remove()
        lines.pop()

    while len(points):
        points[-1].remove()
        points.pop()

    while len(cones):
        cones[-1].remove()
        cones.pop()
        
    while len(vecs):
        vecs[-1].remove()
        vecs.pop()
        
    while len(titles):
        titles[-1].remove()
        titles.pop()

# Frame by frame update of the model
def update_cmodel(frame):
    # Substitute frame for t in functions
    O2_frame = np.asarray([float(i.subs(t, frame)) for i in O2])
    #n1_frame = np.asarray([float(i.subs(t, frame)) for i in n1])
    #n2_frame = np.asarray([float(i.subs(t, frame)) for i in n2])
    #n3_frame = np.asarray([float(i.subs(t, frame)) for i in n3])
    M0_frame = np.asarray([float(i.subs(t, frame)) for i in M0])
    M_frame = np.asarray([float(i.subs(t, frame)) for i in M])
    V_frame = np.asarray([float(i.subs(t, frame)) for i in V])
    A_M_frame = np.asarray([float(i.subs(t, frame)) for i in A_M])

    A_t = np.dot(A_M_frame, V_frame) / np.linalg.norm(V_frame) * V_frame / np.linalg.norm(V_frame)
    A_n = A_M_frame - A_t

    a_scale = 0.003
    v_scale = 0.03

    clear_model()

    # Plot the second cone
    #cones.append(truncated_cone(A0, O2_frame, 0, r, ax))
    
    # Plot the M point
    points.extend([
        ax.scatter(A0[0], A0[1], A0[2], color='red', linewidth=1),
    #    ax.scatter(M_frame[0], M_frame[1], M_frame[2], color='red', linewidth=1),
    #    ax.scatter(M0_frame[0], M0_frame[1], M0_frame[2], color='red', linewidth=1),
    #    ax.scatter(O2_frame[0], O2_frame[1], O2_frame[2], color='red', linewidth=1),
    ])
    
    # Plot the lines for clearer picture
    #lines.extend([ax.plot([M0_frame[0], O2_frame[0]], [M0_frame[1], O2_frame[1]], [M0_frame[2], O2_frame[2]], color='blue', alpha=0.5),
    #             #ax.plot([M0_frame[0], A0[0]], [M0_frame[1], A0[1]], [M0_frame[2], A0[2]], color='green'),
    #             ax.plot([A0[0], O2_frame[0]], [A0[1], O2_frame[1]], [A0[2], O2_frame[2]], color='blue', alpha=0.5),
    #])
    
    # Plot vectors
    vecs.extend([
        ax.quiver(0, 0, 0, V_frame[0], V_frame[1], V_frame[2], color='black', length=v_scale),
        ax.quiver(0, 0, 0, A_M_frame[0], A_M_frame[1], A_M_frame[2], color='green', length=a_scale),
        ax.quiver(0, 0, 0, A_t[0], A_t[1], A_t[2], color='red', length=a_scale),
        ax.quiver(0, 0, 0, A_n[0], A_n[1], A_n[2], color='blue', length=a_scale),
    ])
    
    # Plot titles
    titles.extend([
        ax.text(V_frame[0] * v_scale, V_frame[1] * v_scale, V_frame[2] * v_scale, r'$\vec{V_M}$', fontsize=12),
        ax.text(A_M_frame[0] * a_scale, A_M_frame[1] * a_scale, A_M_frame[2] * a_scale, r'$\vec{a}$', fontsize=12),
        ax.text(A_t[0] * a_scale, A_t[1] * a_scale, A_t[2] * a_scale, r'$\vec{a_t}$', fontsize=12),
        ax.text(A_n[0] * a_scale, A_n[1] * a_scale, A_n[2] * a_scale, r'$\vec{a_n}$', fontsize=12)
    ])
    
    return []


# Generate animation
anim = FuncAnimation(fig, update_cmodel, frames=np.linspace(0, 2, 500),
                        init_func=init_cmodel, blit=True)

anim.save('task2_vecs.gif', dpi=300, writer=PillowWriter(fps=60))
plt.close('all')

In [ ]:
R / r

In [ ]:
OO1

In [ ]:
R / np.sqrt(40 * 40 - OO1**2)

In [ ]:
OO1 / (10 * np.sqrt(2)) * w1

In [ ]:
OO1/np.sqrt(2) / 10

In [ ]:
OA * (np.sqrt(3) + 1) / 4